In [297]:
import pandas as pd
import json

In [298]:
file1_df = pd.read_json('raw_data/purchase_data.json')
file2_df = pd.read_json('raw_data/purchase_data2.json')
print (len(file1_df))
print (len(file2_df))

780
78


In [299]:
#Check to make sure columns are the same between both files
file1_df.columns,file2_df.columns

(Index(['Age', 'Gender', 'Item ID', 'Item Name', 'Price', 'SN'], dtype='object'),
 Index(['Age', 'Gender', 'Item ID', 'Item Name', 'Price', 'SN'], dtype='object'))

In [300]:
pymoli_df = file1_df.append(file2_df)
#pymoli_df = pd.merge(file1_df,file2_df, on="", how="inner")

In [301]:
#player count
playerCount = len(pymoli_df['SN'])     #alternate way: playerCount = pymoli_df['SN'].count().mean()
print(playerCount)

858


In [302]:
#purchase analysis

#number of unique items
uniqueItemCount = len(pymoli_df['Item ID'].unique())

#average purchase price
averagePurchasePrice = pymoli_df["Price"].mean()

#total number of purchases
purchaseCount = pymoli_df['Price'].count() #pymoli_df.loc[pymoli_df['Price'] == 0.00] #search through purchases to make sure non are free/0.00

#total revenue
totalRevenue = pymoli_df['Price'].sum()
#pymoli_df['Price'].value_counts()

print(uniqueItemCount)
print(averagePurchasePrice)
print (purchaseCount)
print (totalRevenue)

summary_df = pd.DataFrame({"UniqueItems":uniqueItemCount})

summary_df["test"] = averagePurchasePrice

184
2.9305710955710915
858
2514.4299999999967


In [303]:
#gender demographics
genderCount = pymoli_df['Gender'].value_counts()
percentageGender = pymoli_df['Gender'].value_counts()/len(pymoli_df)*100
print (genderCount)
print (percentageGender)

Male                     697
Female                   149
Other / Non-Disclosed     12
Name: Gender, dtype: int64
Male                     81.235431
Female                   17.365967
Other / Non-Disclosed     1.398601
Name: Gender, dtype: float64


In [324]:
gender_df_price =  pymoli_df.groupby(['Gender']).mean()["Price"]
gender_df_count =  pymoli_df.groupby(['Gender']).count()["Price"]

#normalized totals = divide total purchase value by count of each gender in dataset
gender_df_price.head()

gender_summary_df = pd.DataFrame({'AveragePrice':gender_df_price,'PurchaseCount':gender_df_count})

Gender
Female                   2.847584
Male                     2.944448
Other / Non-Disclosed    3.155000
Name: Price, dtype: float64